# Castro buses

In [2]:
import pandas as pd
import numpy as np
from openpyxl import Workbook

## Carga de datos desde archivo Excel

In [3]:
# chiloebus
empresas = pd.read_excel('../data/chiloebus.xlsx', sheet_name='CASTRO-empresas')
destinos = pd.read_excel('../data/chiloebus.xlsx', sheet_name='CASTRO-destinos')

In [9]:

empresas.head()

,Operador,Empresa,Rut,ID,Telefono,Nombre_contacto,Correo
0,SERGIO AGUILAR ASENCIO,EXPRESO DEGAÑ,10732404-6,1.0,984667569.0,Sergio Aguilar,sergio.aguilar.asencio1966@gmail.com
1,JAVIERA TUREO CHODIL,BUSES JAVIERA,19723056-8,4.0,996555375.0,Javiera Tureo,javiera.tureo@gmail.com
2,AURELIANO OJEDA GARAY,BUSES OJEDA,10122842-8,5.0,951759186.0,Aureliano Ojeda,busesojeda@gmail.com
3,QUEILEN BUS NORTE LTDA.,QUEILEN BUS,76300596-8,9.0,NaN,NaN,jorge@queilenbus.cl
4,FERMIN BAUTISTA OYARZUN VARGAS,BUSES SAN CRISTOBAL,12760824-5,10.0,998244655.0,NaN,busessancristobal@gmail.com


In [10]:
empresas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64 entries, 0 to 63
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Operador         64 non-null     object 
 1   Empresa          13 non-null     object 
 2   Rut              64 non-null     object 
 3   ID               20 non-null     float64
 4   Telefono         57 non-null     float64
 5   Nombre_contacto  42 non-null     object 
 6   Correo           56 non-null     object 
dtypes: float64(2), object(5)
memory usage: 3.6+ KB


In [12]:
# Establecer 'Rut' como índice en los dataframes existentes
for name in ['empresas', 'destinos']:
    df = globals().get(name)
    if df is None:
        print(f"{name} no está definido en el entorno.")
        continue
    if 'Rut' in df.columns:
        df.set_index('Rut', inplace=True)
    else:
        print(f"'Rut' no está en las columnas de {name}.")

In [17]:
# cambia el nombre de 'Razon Social' a 'Operador' en el dataframe de empresas
#empresas.rename(columns={'Razon Social': 'Operador'}, inplace=True)
destinos.sample(10)

,Hora de Salida,Operador,Empresa,ID,Destino,Andén,Observaciones
Rut,,,,,,,
77197578-K,13:15:00,DISFRUTA SPA,NaN,NaN,DALCAHUE MOCO,3.0,LUNES A DOMINGO
77093001-4,12:10:00,TRANSPORTES SOTOMAYOR,TRANSPORTES SOTOMAYOR,12.0,SAN JUAN,5.0,LUNES A DOMINGO
9692568-9,11:40:00,JUAN ORLANDO DIAZ CARTES,NaN,37.0,ACHAO,NaN,DOMINGO
13169779-1,11:15:00,JOSE LUIS OJEDA OJEDA,NaN,NaN,PUTEMUN,4.0,LUNES A SABADO
10122842-8,08:30:00,AURELIANO OJEDA GARAY,BUSES OJEDA,5.0,CUCAO,5.0,LUNES A DOMINGO
76611546-2,13:20:00,CRISTIAN OYARZUN CARCAMO,NaN,NaN,CHONCHI,2.0,LUNES A VIERNES
11413960-2,11:30:00,JOSE RENE OJEDA ULLOA,NaN,NaN,PUTEMUN,4.0,MIERCOLES
13168478-9,19:30:00,CLAUDIO TRUJILLO TRUJILLO,BUSES TRUJILLO,14.0,RILAN,7.0,LUNES A VIERNES
13169779-1,18:45:00,JOSE LUIS OJEDA OJEDA,NaN,NaN,QUEMCHI,3.0,LUNES A SABADO


In [18]:
# Crear 'sindestinos' con las empresas que no aparecen en 'destinos' (unión por índice 'Rut'). Usamos merge con indicador para identificar filas sin correspondencia en destinos
merged = empresas.merge(destinos, left_index=True, right_index=True, how='left', indicator=True)

# Filtrar solo las empresas sin destinos asociados
left_only = merged[merged['_merge'] == 'left_only']

sindestinos = empresas.loc[left_only.index].copy()

# Mostrar resultado
sindestinos

,Operador,Empresa,ID,Telefono,Nombre_contacto,Correo
Rut,,,,,,
76300596-8,QUEILEN BUS NORTE LTDA.,QUEILEN BUS,9.0,NaN,NaN,jorge@queilenbus.cl
76305760-7,EUROBUS (ETM),ETM,15.0,NaN,NaN,rbohle@busesetm.cl
77642229-0,TRANSPORTE LOS PELLINES LTDA. JORDAN,BUSES JORDAN,25.0,989055953.0,Daniela Villanueva,daniela.villanueva@busesjordan.cl
11118129-2,HERACLIO HECTOR ARRIAGADA OYARZUN,NaN,28.0,NaN,NaN,NaN
6476541-6,JAIME ARNOLDO MUÑOZ CARDENAS,NaN,NaN,NaN,Jaime muñoz,NaN


In [29]:
# crear un exceel con el dataframe sindestinos
sindestinos.to_excel('../data/CASTRO-sindestinos.xlsx', index=True)

In [18]:
# RUTs que aparecen en 'destinos' pero no en 'empresas' -> crear 'sinempresa' con columnas 'Rut' y 'Operador'
mask = ~destinos.index.isin(empresas.index)
sinempresa = destinos.loc[mask, ['Operador']].reset_index().drop_duplicates(subset='Rut').reset_index(drop=True)
sinempresa

,Rut,Operador
0,NaN,CARLOS GOMEZ
